# Main Notebook for Extracting Alcohol Information from PDF Pen Pro Results

In [1]:
import pandas as pd
import numpy as np
import string

import os
import docx
import nltk
from nltk.corpus import words
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = 'C:\\Users\\layto\\AppData\\Local\\Tesseract-OCR\\tesseract'

docxPath = 'C:\\Users\\layto\\SampleWineCatalogs\\Docx_Files'
jpgPath = 'C:\\Users\\layto\\SampleWineCatalogs\\JPG_Files'

In [2]:
len(os.listdir(docxPath))

85

In [3]:
len(os.listdir(jpgPath))

100

In [4]:
temp = [x.split('.')[0][-4:] for x in os.listdir(docxPath)]
jpg_equal_docx = []
for i in range(len(os.listdir(jpgPath))):
    if os.listdir(jpgPath)[i].split('.')[0][-4:] in temp:
        jpg_equal_docx.append(os.listdir(jpgPath)[i])
    

In [5]:
print(len(jpg_equal_docx))
print(jpg_equal_docx[:5])

85
['UCD_Lehmann_0011.jpg', 'UCD_Lehmann_0014.jpg', 'UCD_Lehmann_0047.jpg', 'UCD_Lehmann_0069.jpg', 'UCD_Lehmann_0190.jpg']


-------------------

In [6]:
def whichIsMore(self):
    '''
    Input a list of strings (for example if you split the textual string on whitespace)
    Returns tuple of the counts of '\n' and '\t' respectively to determine which is better to split on.
    '''
    if self and isinstance(self,list) is True:
        counterN = 0
        counterT = 0
        for line in range(len(self)):
            if self[line].find('\n') != -1:
                counterN += 1
            elif self[line].find('\t') != -1:
                counterT += 1
            elif self[line].find('\n') != -1 and self[line].find('\t') != -1:
                counterN += 1
                counterT += 1
        result = (counterN,counterT)
        if result[0] > result[1]:
            return({'\n':result[0],'by':result[0]-result[1]})
        elif result[1] > result[0]:
            return({'\t':result[1],'by':result[1]-result[0]})
        elif result[0] == 0 and result[1] == 0:
            return(None)
    else:
        raise AttributeError('Input a list of strings')

-----

In [7]:
class wordDocx():
    
    def __init__(self, path):
        self.path = path
    
    def fileList(path):
        '''
        Generates full file paths to Docx files given relative directory input
        '''
        import os
        fullPathFiles = []
        file = os.listdir(path)
        for doc in range(len(file)):
            fullPathFiles.append(path + '\\' + file[doc])
        return(fullPathFiles)
    

    def parseTable(file):
        '''
        When getAllParagraphs function below cannot find any text, it checks for instance of a table in the docx file.
        '''
        from docx.api import Document
        document = Document(file)
        
        data = []
        
        table = document.tables
        for tab in range(len(table)):
            oneTable = table[tab]
            for i, row in enumerate(oneTable.rows):
                text = [cell.text for cell in row.cells]
                newText = [' '.join([x for x in text])]
                data.append([' '.join([x.split() for x in newText][0])])

        return([' '.join(x) for x in data])
    
    
    def getAllParaAndTables(files):
        '''
        Extracts all paragraphs and tables from all Docx files of the PDF Pen Pro Results
        '''
        import docx
        if isinstance(files,list) is True:
            docPara = []
            for file in files:
                iterDoc = docx.Document(file)
                docText = []
                if len(''.join([i.text for i in iterDoc.paragraphs])) > 0 and len(iterDoc.tables) > 0:
                    table = wordDocx.parseTable(file)
                    text = ' '.join([i.text for i in iterDoc.paragraphs])
                    makeLines = text.split(' ')
                    if whichIsMore(makeLines) == None:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in text.split()])]},{'TABLE':table}]})
                    else:
                        if list(whichIsMore(makeLines).items())[0][0] == '\n':
                            newLines = text.split('\n')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':table}]})
                        elif list(whichIsMore(makeLines).items())[0][0] == '\t':
                            newLines = text.split('\t')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':table}]})
    
                elif len(''.join([i.text for i in iterDoc.paragraphs])) == 0 and len(iterDoc.tables) > 0:              
                    table = wordDocx.parseTable(file)
                    if len(table) > 0:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':None},{'TABLE':table}]})
                    else:
                        return(None)
                elif len(''.join([i.text for i in iterDoc.paragraphs])) > 0 and len(iterDoc.tables) == 0:
                    text = ' '.join([i.text for i in iterDoc.paragraphs])
                    makeLines = text.split(' ')
                    if whichIsMore(makeLines) == None:
                        docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in text.split()])]},{'TABLE':None}]})
                    else:
                        if list(whichIsMore(makeLines).items())[0][0] == '\n':
                            newLines = text.split('\n')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':None}]})
                        elif list(whichIsMore(makeLines).items())[0][0] == '\t':
                            newLines = text.split('\t')
                            docPara.append({file.split('\\')[-1]:[{'TEXT':[' '.join([x for x in newLines])]},{'TABLE':None}]})
                        
                elif ''.join([i.text for i in iterDoc.paragraphs]) == '' and len(iterDoc.tables) == 0:
                    docPara.append({file.split('\\')[-1]:None})

            return(docPara)

        else:
            raise AttributeError('Needs list of full file paths to the .docx files')

---------

In [68]:
class identifyFormat():
    
    def __init__(self, pargraph):
        self.paragraph = paragraph
        
    def years(mini,maxi):
        '''
        Generates a set of guess year ranges for alcohols
        '''
        import numpy as np
        years = set([n for n in np.arange(mini,maxi,1)])
        return(years)
    
    def wordsAndNums(self):
        '''
        checks word for letters and nums
        '''
        import string
        import numpy as np
        
        setLetters = set(list(string.ascii_lowercase))
        setNums = set([str(x) for x in np.arange(0,10,1)])
        words = set(list(self.lower())).intersection(setLetters)
        nums = set(list(self)).intersection(setNums)
        if len(words) > 1 and len(nums) > 1:
            return('both')
        else:
            return('fine')
        
    def fixLettersNums(self):
        '''
        fixed string for letters and nums together
        '''
        import string
        import numpy as np

        setLetters = set(list(string.ascii_lowercase))
        setNums = set([str(x) for x in np.arange(0,10,1)])
        l = list(self)
        nums = []
        letters = []
        for x,y in enumerate(l):
            if l[x] in setNums:
                nums.append(x)
            elif l[x] in setLetters:
                letters.append(x)
        return(nums,letters)
    
    def splitWord(self):
        '''finds the split of conjoined words'''
        import nltk
        # UNCOMMENET BELOW IF NOT INSTALLED
        #nltk.download('words')
        from nltk.corpus import words
        import string
        split = self.split()
        for i in range(len(split)):
            l = list(split[i])
            upper = [x for x in list(split[i]) if x.isupper() is True]
            if len(upper) == 1:
                word1 = ''.join(l[:l.index(upper[0])]).lower()
                word2 = ''.join(l[l.index(upper[0]):]).lower()
                if len(word1) > 1 and len(word2) > 1:
                    if word1[-1] != 's' and word2[-1] != 's':
                        if word1 in words.words() and word2 in words.words():
                            split[i] = (word1 + ' ' + word2)
                        else:
                            if word1 in words.words() and word2 not in words.words():
                                split[i] = word1
                            elif word1 not in words.words() and word2 in words.words():
                                split[i] = word2
                    else:
                        if word1 in words.words() or word2 in words.words():
                            split[i] = (word1 + ' ' + word2)
        return(' '.join(split))
            
    def removeSpecial(self):
        '''
        Cleans the strings from any special or unnecessary characters
        '''
        special = set([',','&','"','|','\\',':',';','`','~','#','°','•',"'",'*','/','_','—'])
        split = self.split(' ')
        split = [x for x in split if len(x) > 0]
        fixed = []
        
        for i in range(len(split)):
            test = list(split[i])
            if '(' in test and '.' in test:
                split[i] = ''.join([x for x in test if x != '(' and x != ')' and x != '.'])
                fixed.append(split[i])
            elif len(set(test).intersection(special)) == 0 and '.' not in test:
                if '(' or ')' in test:
                    try:
                        if test[-2] == '(' and test[-1] == ')':
                            split[i] = ''.join(test[:-2])
                            fixed.append(split[i])
                        elif test[0] == '(' and test[1] != ')' and test[-1] != ')':
                            split[i] = ''.join(test[1:])
                            fixed.append(split[i])
                        elif test[0] != '(' and test[1] != ')' and test[-1] == ')':
                            split[i] = ''.join(test[:-1])
                            fixed.append(split[i])
                        elif test[0] == '(' and test[1] == ')':
                            split[i] = ''.join(test[2:])
                            fixed.append(split[i])
                        elif test[0] == '(' and test[-1] == ')':
                            split[i] = ''.join(test[1:-1])
                            fixed.append(split[i])
                        else:
                            fixed.append(split[i])
                    except IndexError:
                        continue
                elif '-' in test:
                    fixed.append(split[i].replace('-',' '))
                else:
                    fixed.append(split[i])
            elif test[-1] == '.' and test[0] != '.' and len(test) > 1:
                test = test[:-1]
                split[i] = ''.join(test)
                fixed.append(split[i])
            elif '.' in test:
                if len([x for x in test if x == '.']) >= 2 and test[0] != '.':
                    try:
                        finding = [x.find('.') for x in test]
                        for i in range(len(finding)):
                            if finding[i+1] == 0 and finding[i] == 0:
                                badInd = i
                                if badInd > 1:
                                    fixed.append(''.join(test[:i]))
                                elif badInd <= 1: 
                                    fixed.append(''.join(test[i+2:]))
                    except IndexError:
                        continue
                elif len([x for x in test if x == '.']) == 1:
                    try:                        
                        if isinstance(int(''.join([x for x in test if x != '.'])),int):
                            split[i] = ''.join(test)
                            fixed.append(split[i])
                        else:
                            split[i] = ''.join([x for x in test if x != '.'])
                            fixed.append(split[i])
                            
                    except ValueError:
                        continue
                
                # For multiple leading '.' before prices
                elif len([y for y in [x.find('.') for x in test] if y == 0]) != 1 and test[0] == '.':
                    tryInt1 = ''.join(test).split('.')[-1]
                    tryInt2 = ''.join(test).split('.')[-2]
                    try:
                        if isinstance(int(tryInt1),int) and isinstance(int(tryInt2),int):
                            if test[0] != '.':
                                joinSplit = ''.join(test).split('.')
                                name = joinSplit[0]
                                prices = joinSplit[-2] + '.' + joinSplit[-1]
                                split[i] = name + ' ' + prices
                                fixed.append(split[i])
                            elif test[0] == '.':
                                joinSplit = ''.join(test).split('.')
                                price = joinSplit[-2] + '.' + joinSplit[-1]
                                split[i] = price
                                fixed.append(split[i])
                    except ValueError:
                        length = len([y for y in [x.find('.') for x in test] if y == 0])-1
                        test = test[length:]
                        split[i] = ''.join(test)
                        fixed.append(split[i])
            else:
                while len(set(test).intersection(special)) > 0:
                    ind = [test.index(x) for x in set(test).intersection(special)]
                    if len(ind) > 0:
                        try:
                            for index in ind:
                                test.pop(index)
                        except IndexError:
                            continue
                    split[i] = ''.join(test)
                    if len(set(split[i]).intersection(special)) == 0:
                        fixed.append(split[i])
                        break
                    else:
                        continue
                
        fixed = ' '.join([''.join(x) for x in fixed])
        return(fixed)
    
    
    def fixBottlePrices(self):
        '''
        Removes special using function above, and fixes bad bottle prices
        '''
        fixed = identifyFormat.removeSpecial(self)
        again = identifyFormat.splitWord(fixed)
        
        preSplit = again.split(' ')
        split = [x for x in preSplit if len(x) > 0]
        
        setRange = np.arange(1,5000,1)
        
        for i in range(len(split)):
            if '.' in list(split[i]) and i >= 2:
                tryInt = split[i-1]
                intList = list(tryInt)
                try:
                    if isinstance(int(tryInt),int):
                        getLength = len(intList)
                        if getLength == 1:
                            split[i-1] = intList[0]+'.'+'00'
                        elif getLength == 2 and split[i-2].lower().find('no') == -1:
                            split[i-1] = intList[0]+'.'+intList[1]+'0'
                        elif getLength == 3:
                            split[i-1] = intList[0]+'.'+intList[1]+intList[2]
                        elif getLength == 4 and int(tryInt) not in np.arange(1880,2000,1):
                            split[i-1] = intList[0]+intList[1]+'.'+intList[2]+intList[3]
                        else:
                            split[i-1] = split[i-1]
                except ValueError:
                    continue
                    
        for i in range(len(split)+1):
            try:
                single = split[i]
                testNO = split[i-1]
                try:
                    if int(single) in np.arange(1,10,1):
                        if testNO.lower() != 'no':
                            ind = split.index(single)
                            split.pop(int(ind))
                except ValueError:
                    continue
            except IndexError:
                break

        return(' '.join(split))
    
                                
    def findYears(self,mini,maxi):
        '''
        finds year(s) in fixed word docx paragraphs associated with respective alcohol
        '''
        yearSet = identifyFormat.years(mini,maxi)
        fixed = identifyFormat.removeSpecial(self)
        split = fixed.split(' ')
        y = []
        for i,j in enumerate(split):
            try:
                year = int(split[i])
                if year and year in yearSet:
                    y.append({str(year):i})
            except ValueError:
                continue
                
        if len(y) > 0:
            return(y)
        else:
            return(0)

    def wordBottle(self):
        '''
        From string of paragraph as input, finds instance of word 'bottle'
        '''
        try:
            test = [x.lower() for x in self.split(' ')]
            find = test.index('bottle')
            if find:
                return(True)
        except ValueError:
            return(False)
        
    def idNum(self):
        '''
        Generates a set of bin ranges for alcohols
        '''
        fixed = self.split(' ')
        import numpy as np
        set1 = set([n for n in np.arange(1,1880,1)])
        set2 = set([n for n in np.arange(2005,15000,1)])
        ID = set(set1.union(set2))
        potentialIDs = []

        for i in fixed:
            try:
                if int(i) in ID:
                    ind = fixed.index(i)
                    potentialIDs.append({i:ind})
            except ValueError:
                continue
            
        if len(potentialIDs) > 0:
            return(potentialIDs)
        elif len(potentialIDs) == 0:
            return('')

----

In [9]:
allData = wordDocx.getAllParaAndTables(wordDocx.fileList(docxPath))

In [10]:
# Cleans "allData" with identifyFormat class

files = wordDocx.fileList(docxPath)
for i in range(len(allData)):
    fileName = files[i].split('\\')[-1]
    if allData[i][fileName] != None:
        preText = allData[i][fileName][0]['TEXT']
        preTable = allData[i][fileName][1]['TABLE']
        if preText != None and preTable != None:
            preText = preText[0]
            preTab = ' '.join([x for x in preTable])
            newText = identifyFormat.fixBottlePrices(preText)
            newTable = identifyFormat.fixBottlePrices(preTab)
            allData[i][fileName][0]['TEXT'] = [newText]
            allData[i][fileName][1]['TABLE'] = [newTable]
        elif preText == None and preTable != None:
            preTable = ' '.join([x for x in preTable])
            newTable = identifyFormat.fixBottlePrices(preTable)
            allData[i][fileName][1]['TABLE'] = [newTable]
        elif preText != None and preTable == None:
            preText = preText[0]
            newText = identifyFormat.fixBottlePrices(preText)
            allData[i][fileName][0]['TEXT'] = [newText]
    else:
        allData[i][fileName] == None

In [67]:
print('Number of word docs: ' + str(len(allData)))
allData[0]

Number of word docs: 85


{'UCD_Lehmann_0011.docx': [{'TEXT': ['The Great 1961 GLAR ET8 We quote from the new wine classic the Time-Life book on Wines and Spirits 1961 RED BORDEAUX-one of the best years of the century the equal of 1945 Unfortunately the crop was about half the normal size The grapes were well-nourished resulting in concentrated wines rich in every quality including color body bouquet and fruit They are the most long-lived wines of our generation in the classic tradition 20 out of possible 20 very very great" There is little that we can add to this other than to say that the 1961s have gone up 40% in Bordeaux during the last year if one is fortunate enough to be able to obtain any of the great ones at all 1961 VINTAGE No Bottle Case 237 CHATEAU BELAIR St Emilion 4.49 48.50 There are 12 First Great Growths in the 1955 St Emilion official classification Chateau Belair is one of them The vineyard is owned by Chateau Ausone the vines are adjacent and the wine is matured in the Ausone cellar The dist

In [58]:
# Grabbing tester to test functions from the table of UCD_Lehmann_3028.docx
for i in range(len(allData)):
    try:
        if allData[i]['UCD_Lehmann_3028.docx']:
            t = allData[i]['UCD_Lehmann_3028.docx']
            _3028 = t[0]['TEXT'][0]
            break
    except KeyError:
        continue

In [59]:
_3028

'19TH CENTURY OLD VINTAGE MADEIRAS About year ago we learned that there was small treasury of Vintage Madeira in the cellars of Blandy in Funchal that dated as far back as the late 18th Century We were aware of the importance of this wine among the wine drinkers of early American history we had tasted old examples and knew how extraordinarily fine and vigorous they could be and we resolved that we would acquire some for our clientele It took year of much persuasion to accomplish the mission and at long last they are here in all their antiquity and glory Here is an epoch-making collection of the world oldest and most long-lived wines wines that are true collectors items as surely as fine first edition of rare old book We list below the total number of bottles that are in existence in the world the vintage year and every drop is of the year stated) and the price per bottle Quite frankly we have no precedent to gauge the rate of depletion but if you are interested in any particular vintag

In [60]:
# Grabbing tester to test functions from the table of UCD_Lehmann_0800.docx 
for i in range(len(allData)):
    try:
        if allData[i]['UCD_Lehmann_0800.docx']:
            t2 = allData[i]['UCD_Lehmann_0800.docx']
            _0800 = t2[1]['TABLE'][0]
            break
    except KeyError:
        continue

In [61]:
tester_0800 = identifyFormat.fixBottlePrices(_0800)
tester_0800

'ADVANCE SALE OFFERING OF 1970 URINES MOSELLES EVERYONE IS TALKING about 1970 on the Rhine and Moselle We have gone to great effort to see that the 70s would be shipped express so they would be here in time to add their freshness fruit and good cheer to your summer pleasures All of the 1970s shown below will reach our cellars the first few days of July and you can count on delivery before July 10th whether it be your apartment in the city or your summer vacation address The growers along the Rhine and Moselle rivers are rightfully happy with their 1970 crop It is true that they did not produce too many peak wines in the exalted Auslese and Beerenauslese classhowever this was more than compensated for by bounteous crop of truly superb typical and well-balanced regionals and the less complex estatebottlings After all it is these less costly and less sweet German wines that have proven most popular in America All of these represent regional wines at their best and come from an impeccable 

In [62]:
identifyFormat.findYears(_3028,1800,2000)

[{'1957': 227},
 {'1952': 261},
 {'1949': 275},
 {'1952': 280},
 {'1952': 285},
 {'1949': 321}]

In [63]:
[list(x.keys())[0] for x in identifyFormat.idNum(_3028)]

['1792', '679', '21']

------

## First finalized method for extacting names from tables/paragraphs that use the bottle numbers, name, year (maybe), and price format.

In [18]:
def priceIndexing(para):
    '''
    Finds instances of prices
    '''
    text = para.split(' ')
    indices = []
    for i,j in enumerate(text):
        try:
            if j.find('.') != -1 and isinstance(int(''.join(j.split('.'))),int):
                indices.append(i)
        except ValueError:
            continue

    final = []
    ind1 = 0
    if len(indices) == 2 and indices[1]-indices[0] == 1:
        final.append(' '.join(text))
    else:
        for r in range(len(indices)):
            try:
                if indices[r+1] - indices[r] == 1 and indices[r+2] + indices[r+1] - indices[r] != 1:
                    ind2 = indices[r+1] + 1
                    final.append(' '.join(text[ind1:ind2]))
                    ind1 = ind2
                elif indices[r+1] - indices[r] != 1:
                    ind2 = indices[r] + 1
                    final.append(' '.join(text[ind1:ind2]))
                    ind1 = ind2
            except IndexError:
                continue
    return([x for x in final if x != ''])
    
def finalForm(para):
    '''
    input is a textual paragraph preprocessed using the identifyFormat.fixBottlePrices() method which includes removing 
    problematic symbols or characters (via identifyFormat.removeSpecial())
    '''
    findYears = identifyFormat.findYears(para,1880,2000)
    findIDs = identifyFormat.idNum(para)
    
    newFinal = []
    final = priceIndexing(para)

    if len(findIDs) > 0: #and len(findIDs) == len(findYears):
        for i,j in enumerate(final):
            if len(j.split()) > 1:
                if identifyFormat.idNum(j) != '' and len(identifyFormat.idNum(j)) == 1 and len(j.split()) < 27:
                    newFinal.append(j)
                elif identifyFormat.idNum(j) != '' and len(identifyFormat.idNum(j)) == 1 and len(j.split()) >= 27:
                    for t in range(len(j.split())):
                        try:
                            if j.split()[-[t+1][0]].isupper() is True and j.split()[-[t+1][0]-1].isupper() is False:
                                ind = j.split().index(j.split()[-[t+1][0]])
                                j = ' '.join(j.split()[ind:])
                                newFinal.append(j)
                            else:
                                continue
                        except IndexError:
                            continue
                elif identifyFormat.idNum(j) != '' and len(identifyFormat.idNum(j)) > 1 and 'and' not in j.split():
                    length = len(identifyFormat.idNum(j))
                    c = 0
                    splitSentence = j.split()
                    for i in range(len(splitSentence)):
                        if splitSentence[i] in [list(x.keys())[0] for x in identifyFormat.idNum(j)]:
                            c += 1
                            if c == length:
                                ind = j.split().index(j.split()[i])
                                j = ' '.join(j.split()[ind:])
                                newFinal.append(j)
                elif identifyFormat.idNum(j) == '':
                    for t in range(len(j.split())):
                        try:
                            if j.split()[-[t+1][0]].isupper() is True and j.split()[-[t+1][0]-1].isupper() is False:
                                ind = j.split().index(j.split()[-[t+1][0]])
                                j = ' '.join(j.split()[ind:])
                                newFinal.append(j)
                            else:
                                continue
                        except IndexError:
                            continue
    elif findIDs == '':
        for i,j in enumerate(final):
            if len(j.split()) > 1:
                for t in range(len(j.split())):
                    try:
                        if j.split()[-[t+1][0]].isupper() is True and j.split()[-[t+1][0]-1].isupper() is False:
                            ind = j.split().index(j.split()[-[t+1][0]])
                            if j.split()[ind-1] in [str(x) for x in np.arange(1,10000,1)]:
                                j = ' '.join(j.split()[ind-1:])
                                newFinal.append(j)
                            else:
                                j = ' '.join(j.split()[ind:])
                                newFinal.append(j)
                        else:
                            continue
                    except IndexError:
                        continue
        

    n = [str(x) for x in np.arange(0,1000,1)]
    for f in range(len(newFinal)):
        if newFinal[f].split()[0] not in n:
            ind = [newFinal[f].split().index(x) for x in newFinal[f].split() if x in n]
            if len(ind) > 0:
                reconstruct = ' '.join(newFinal[f].split()[ind[0]:])
                newFinal[f] = reconstruct

    if len(newFinal) > 0:
        return(newFinal)
    else:
        return('')


In [19]:
finalForm('SALE PRICE No DI Bottle Case 1958 OLD JEB YEAR OLD KENTUCKY BOURBON 3.69 43.50')

['1958 OLD JEB YEAR OLD KENTUCKY BOURBON 3.69 43.50']

In [20]:
resultBinNameYearPrices = {}
            
files = wordDocx.fileList(docxPath)
for i in range(len(allData)):
    fileName = files[i].split('\\')[-1]
    if allData[i][fileName] != None:
        preText = allData[i][fileName][0]['TEXT']
        preTable = allData[i][fileName][1]['TABLE']
        if preText != None and preTable != None:
            preText = preText[0]
            preTab = ' '.join([x for x in preTable])
            textResult = finalForm(preText)
            tableResult = finalForm(preTab)
            if textResult != None and tableResult != None:
                if textResult != '' and tableResult != '':
                    resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':tableResult}]
                elif textResult == '' and tableResult != '':
                    resultBinNameYearPrices[fileName] = [{'textResult':None},{'tableResult':tableResult}]
                elif textResult != '' and tableResult == '':
                    resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':None}]
            
        elif preText == None and preTable != None:
            preTab = ' '.join([x for x in preTable])
            tableResult = finalForm(preTab)
            if tableResult != None and tableResult != '':
                resultBinNameYearPrices[fileName] = [{'textResult':None},{'tableResult':tableResult}]
        
        elif preText != None and preTable == None:
            preText = preText[0]
            textResult = finalForm(preText)
            if textResult != None and textResult != '':
                resultBinNameYearPrices[fileName] = [{'textResult':textResult},{'tableResult':None}]


### Grab the results from training sets we have created

In [54]:
# Showing one results
print('Number of Documents w/results: '+str(len(resultBinNameYearPrices)))
list(resultBinNameYearPrices.items())[32]

Number of Documents w/results: 43


('UCD_Lehmann_3392.docx',
 [{'textResult': None},
  {'tableResult': ['ITALIAN WINES THE BEST OF THE COUNTRY Bottle Case ase 8.25',
    '624 VALPOLICELLA 1961 Cantina Sociale Red 1.69 18.25',
    '619 TYROLEAN RED AUSLESE 1963 1.89 20.40',
    '629 TYROLEAN WHITE AUSLESE 1963 1.89 20.40',
    '1205 GRAHAMS WESTMINSTER TAWNY PORT 1.99 21.95',
    '1268 BLANDYS SERCIAL MADEIRA Dry 2.19 24.50',
    '1269 BLANDYS BUAL MADEIRA Rich 2.19 24.50',
    'VINTAGE 6.90',
    '301 PINOT CHARDONNAY LAMARTINE Caves Cooperative 1.59 17.15',
    '821 MEURSAULT Estate bottled Joseph Matrot 2.59 28.00',
    '345 POUILLY FUISSE Estate bottled Plumet 2.99 32.30',
    '310 CHASSAGNE MONTRACHET LES RUCHOTTES Claude Ramonet 3.99 43.10',
    '295 PULIGNY MONTRACHET Estate bottled Julien Monnot 4.49 48.50',
    '344 MEURSAULT CLOS DES PERRIERES Estate bottled A Grivault 4.59 49.55',
    '369 LE MONTRACHET Estate bottled Milan-Mathey 8.40 91.80',
    '390 MUSIGNY BLANC Estate bottled Comte de Vogue 12.50 137.50',

In [55]:
textual = []
textOnly = [x[1][0]['textResult'] for x in list(resultBinNameYearPrices.items()) if x[1][0]['textResult'] != None]
for i in range(len(textOnly)):
    [textual.append(x) for x in textOnly[i]]

table = []
tableOnly = [x[1][1]['tableResult'] for x in list(resultBinNameYearPrices.items()) if x[1][1]['tableResult'] != None]
for i in range(len(tableOnly)):
    [table.append(x) for x in tableOnly[i]]

allResults = textual + table


print('RESULTS: '+ str(len(allResults)))
allResults[:10]

RESULTS: 460


['CHATEAU LYNCH SAGES Pauillac 4.79 51.75',
 '288 CHATEAU BRANE CANTENAC Margaux 5.09 55.00',
 'CHATEAU LEO VILLE LASCASES St Julien 5.49 59.30',
 '804 CHATEAU CANON St Emilion 5.49 59.30',
 '615 CHATEAU BEYCHEVELLE St Julien 5.99 64.70',
 'CHATEAU CALON SEGUR St Estephe 5.99 64.70',
 'CHATEAU LA MISSION HAUT BRION Graves 9.49 102.50',
 '609 CHATEAU AUSONE St Emilion) 10.95 118.00',
 '207 CHATEAU MARGAUX Margaux 13.50 145.80',
 '158 CHATEAU HAUT BRION Graves 14.50 156.60']

In [23]:
def formatResults(initialList):
    format1 = [x for x in initialList if len(x.split()) > 3]
    format1 = [''.join([y.replace('-',' ') for y in list(format1[i])]) for i in range(len(format1))]
    format1 = [x for x in format1 if 'choice' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'your' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'velvety' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'recommended' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'reinhold' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'fuss' not in list([y.lower() for y in x.split()])]
    format1 = [x for x in format1 if 'distillery' not in list([y.lower() for y in x.split()])]

    format1 = [x for x in format1 if not len(x.split()[-2].split('.')[0]) == len(x.split()[-1].split('.')[0])]           
    # Fixing entry with SALE PRICE BOTTLE CASE words in them text in it:
    tempString = [x for x in format1 if 'bottle' and 'case' in list([y.lower() for y in x.split()])]
    if len(tempString) > 0:
        badWords = [y for y in [x for x in tempString[0].split() if x.lower() in words.words()] if y.isupper() is False]
        indicesBad = [tempString[0].split().index(x) for x in badWords]
        for i in range(len(tempString)):
            format1[format1.index(tempString[i])] = ' '.join([x for x in tempString[i].split() if x not in tempString[i].split()[min(indicesBad):max(indicesBad)+1]])
        return(initialList)
    else:
        return(format1)

In [24]:
def makeDataFrame(listResults):
    
    bottleNo = []
    names = []
    bottlePrice = []
    casePrice = []

    for i in range(len(listResults)):
        split = listResults[i].split()
        if split[0] in [str(x) for x in np.arange(1,5000,1)]:
            if split[-2] != split[-1] and split[-1].find('.') >= 0 and (split[-2].find('.') == 1 or split[-2].find('.') == 0):
                bottleNo.append(int(split[0]))
                names.append(' '.join(split[1:-2]))
                bottlePrice.append(split[-2])
                casePrice.append(split[-1])
            elif split[-1].find('.') >= 0 and len(set([x.lower() for x in list(split[-2])]).intersection(set(list(string.ascii_lowercase))))>1:
                bottleNo.append(int(split[0]))
                names.append(' '.join(split[1:-1]))
                if split[-1].find('.') == 0 or split[-1].find('.') == 1:
                    bottlePrice.append(split[-1])
                    casePrice.append('None')    
                elif split[-1].find('.') == 2:
                    bottlePrice.append('None')
                    casePrice.append(split[-1])

        elif split[0] not in [str(x) for x in np.arange(1,5000,1)]:
            if split[-2] != split[-1] and split[-1].find('.') >= 0 and (split[-2].find('.') == 1 or split[-2].find('.') == 0):
                bottleNo.append('None')
                names.append(' '.join(split[0:-2]))
                bottlePrice.append(split[-2])
                casePrice.append(split[-1])
            elif split[-1].find('.') >= 0 and len(set([x.lower() for x in list(split[-2])]).intersection(set(list(string.ascii_lowercase))))>1:
                bottleNo.append('None')
                names.append(' '.join(split[0:-1]))
                if split[-1].find('.') == 0 or split[-1].find('.') == 1:
                    bottlePrice.append(split[-1])
                    casePrice.append('None')    
                elif split[-1].find('.') == 2:
                    bottlePrice.append('None')
                    casePrice.append(split[-1])
    s1 = pd.Series(bottleNo)
    s2 = pd.Series(names)
    s3 = pd.Series(bottlePrice)
    s4 = pd.Series(casePrice)
    df = pd.concat([s1,s2,s3,s4],axis=1)
    df.columns = ['BottleNo','Name','BottlePrice','CasePrice']
    df = df.sort_values(by='BottlePrice',ascending=True).reset_index().drop(columns=['index'])
    return(df)

In [25]:
_0237 = resultBinNameYearPrices['UCD_Lehmann_0237.docx'][1]['tableResult']
_0237 = formatResults(_0237)
PenPro_0237 = makeDataFrame(_0237)
PenPro_0237.to_csv('PenPro_0237.csv',sep='\t',index=False)


In [26]:
_1198 = resultBinNameYearPrices['UCD_Lehmann_1198.docx'][0]['textResult']
_1198 = formatResults(_1198)
PenPro_1198 = makeDataFrame(_1198)
PenPro_1198.to_csv('PenPro_1198.csv',sep='\t',index=False)

In [27]:
_0629 = resultBinNameYearPrices['UCD_Lehmann_0629.docx'][1]['tableResult']
_0629 = formatResults(_0629)
PenPro_0629 = makeDataFrame(_0629)
PenPro_0629.to_csv('PenPro_0629.csv',sep='\t',index=False)

In [28]:
_2351 = resultBinNameYearPrices['UCD_Lehmann_2351.docx'][0]['textResult']
_2351 = formatResults(_2351)
PenPro_2351 = makeDataFrame(_2351)
PenPro_2351.to_csv('PenPro_2351.csv',sep='\t',index=False)

In [626]:
# good = ['_3479', '_0011', '_0237', '_0551', '_0558', '_0644', '_0869']

In [29]:
_4086 = resultBinNameYearPrices['UCD_Lehmann_4086.docx'][0]['textResult'] + resultBinNameYearPrices['UCD_Lehmann_4086.docx'][1]['tableResult']
_4086 = formatResults(_4086)
PenPro_4086 = makeDataFrame(_4086)
PenPro_4086.to_csv('PenPro_4086.csv',sep='\t',index=False)

In [56]:
PenPro_0237

,BottleNo,Name,BottlePrice,CasePrice
0,753,CHATEAU GISCOURS Margaux New ownership-vast im...,2.79,30.75
1,713,CHATEAU LASCOMBES Margaux,2.99,32.95
2,706,CHATEAU NENIN Pomerol,2.99,32.95
3,757,CHATEAU MALESCOT Margaux,2.99,32.95
4,261,CHATEAU CLOS FOURTET St Emilion,2.99,32.95
5,None,CHATEAU BRANAIRE DUCRU St Julien,2.99,32.95
6,140,CHATEAU LEOVILLE LASCASES St Julien,2.99,32.95
7,107,CHATEAU HAUT BATAILLEY Pauillac,2.99,32.95
8,197,CHATEAU LA POINTE Pomerol,2.99,32.95
9,751,CHATEAU DUCRU BEAUCAILLOU St Julien,3.29,36.25
